In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### InceptionV3

In [3]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
#last_layer = pre_trained_model.get_layer('mixed10')

print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/train/'
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0/255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))


history = model.fit_generator(
            train_generator,
            #validation_data=validation_generator,
            epochs=20,
            verbose=1)


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


<ipython-input-3-b5f8343cbf1d>:89: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
52/52 [==============================] - 23s 193ms/step - loss: 1.1542 - acc: 0.9241
Epoch 2/20
52/52 [==============================] - 10s 189ms/step - loss: 0.1204 - acc: 0.9776
Epoch 3/20
52/52 [==============================] - 10s 187ms/step - loss: 0.0730 - acc: 0.9825
Epoch 4/20
52/52 [==============================] - 10s 188ms/step - loss: 0.0727 - acc: 0.9825
Epoch 5/20
52/52 [==============================] - 10s 188ms/step - loss: 0.0530 - acc: 0.9873
Epoch 6/20
52/52 [==============================] - 10s 189ms/step - loss: 0.0449 - acc: 0.9942
Epoch 7/20
52/52 [==============================] - 10s 188ms/step - loss: 0.0473 - acc: 0.9922
Epoch 8/20
52/52 [==============================] - 10s 189ms/step - loss: 0.0213 - acc: 0.9961
Epoch 9/20
52/52 [==============================] - 10s 188ms/step - loss: 0.0401 - acc: 0.9903
Epoch 10/20
52/52 [==============================] - 10s 187ms/step - loss: 0.0113 - acc: 0.9990
Epoch 11/20
52/52 [====================

### ResNet50

In [4]:
'''
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import RMSprop

#weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
#weights_file = "inception_v3.h5"
#urllib.request.urlretrieve(weights_url, weights_file)
#
#pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
#                                include_top=False,
#                                weights=None)
#pre_trained_model.load_weights(weights_file)

pre_trained_model = ResNet50( input_shape=(150,150,3),
                                  include_top=False,
                                  weights='imagenet'
                            )



for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

#last_layer = pre_trained_model.get_layer('mixed7')
#last_layer = pre_trained_model.get_layer('mixed10')
last_layer = pre_trained_model.get_layer('conv5_block3_out')

print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/train/'
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0/255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))


history = model.fit_generator(
            train_generator,
            # validation_data=validation_generator,
            epochs=20,
            verbose=1)
'''

'\nimport urllib.request\nimport os\nimport zipfile\nfrom tensorflow.keras.preprocessing.image import ImageDataGenerator\nfrom tensorflow.keras import layers\nfrom tensorflow.keras import Model\n#from tensorflow.keras.applications.inception_v3 import InceptionV3\nfrom tensorflow.keras.applications import ResNet50\nfrom tensorflow.keras.optimizers import RMSprop\n\n#weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"\n#weights_file = "inception_v3.h5"\n#urllib.request.urlretrieve(weights_url, weights_file)\n#\n#pre_trained_model = InceptionV3(input_shape=(150, 150, 3),\n#                                include_top=False,\n#                                weights=None)\n#pre_trained_model.load_weights(weights_file)\n\npre_trained_model = ResNet50( input_shape=(150,150,3),\n                                  include_top=False,\n                                  weights=\'imagenet\'\n                            )\n\n\n\nfor 

In [5]:
import numpy as np
from google.colab import files
#from keras.preprocessing import image
import keras.utils as image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving 6d77ed8175e10908144b3af2a48173d9_t.jpeg to 6d77ed8175e10908144b3af2a48173d9_t.jpeg
Saving gahag-0037384573-1.jpg to gahag-0037384573-1.jpg
Saving images.jpeg to images.jpeg
Saving images22.jpeg to images22.jpeg
Saving images33.jpeg to images33.jpeg
Saving img_os06.jpg to img_os06.jpg
Saving publicdomainq-0032819ktf.jpg to publicdomainq-0032819ktf.jpg
1/1 [==============================] - 2s 2s/step
[[1.]]
[1.]
6d77ed8175e10908144b3af2a48173d9_t.jpeg is a human
1/1 [==============================] - 0s 26ms/step
[[1.]]
[1.]
gahag-0037384573-1.jpg is a human
1/1 [==============================] - 0s 26ms/step
[[1.]]
[1.]
images.jpeg is a human
1/1 [==============================] - 0s 29ms/step
[[1.]]
[1.]
images22.jpeg is a human
1/1 [==============================] - 0s 26ms/step
[[1.]]
[1.]
images33.jpeg is a human
1/1 [==============================] - 0s 25ms/step
[[1.]]
[1.]
img_os06.jpg is a human
1/1 [==============================] - 0s 27ms/step
[[1.]]
[1.]
publicdomain